Лабораторная 5.1
1. Пол (sex): отфильтровать строки, где пол не указан, преобразовать оставшиеся в число 0/1;
2. Номер ряда в зале (row_number): заполнить вместо NAN максимальным значением ряда;
3. Количество выпитого в литрах (liters_drunk): отфильтровать отрицательные значения и нереально большие значения (выбросы). Вместо них заполнить средним.

In [76]:
import numpy as np
import pandas as pd

df = pd.read_csv('titanic_with_labels.csv', sep=' ', index_col=0)

df = df[(df['sex'] == 'Мужчина') | (df['sex'] == 'M') | (df['sex'] == 'м') | (df['sex'] == 'Ж') | (df['sex'] == 'ж')]
df.reset_index(drop=True, inplace=True)
df['sex'].replace({'Мужчина': 0, 'M': 0, 'м': 0, 'Ж': 1, 'ж': 1}, inplace=True)
df['row_number'].fillna(df['row_number'].max(), inplace=True)
df.loc[(df['liters_drunk'] < 0) | (df['liters_drunk'] > 5), 'liters_drunk'] = np.nan
df['liters_drunk'].fillna(df['liters_drunk'].mean(), inplace=True)

print(df.head(50))

    sex  row_number  liters_drunk  age        drink  check_number  label
0     1        79.0      2.000000   66         Cola          3116      0
1     0        23.0      0.000000   10        Fanta          8118      0
2     0        92.0      0.000000   18        Fanta          2264      1
3     1        52.0      5.000000   67     Beerbeer          7640      1
4     1        57.0      2.000000   27     Beerbeer          8764      0
5     0        15.0      1.000000   44      Bugbeer          6235      0
6     1        49.0      0.000000   11         Cola           185      1
7     0        33.0      5.000000   64        Fanta          2970      1
8     1         5.0      0.000000   39        Water          6494      0
9     0        48.0      1.000000   64        Water          9971      1
10    1        78.0      2.476708   51        Water          3592      0
11    0        32.0      2.000000   35  Strong beer          2021      0
12    1        12.0      3.000000   59         Cola

Лабораторная 5.2
    4. Возраст (age): разделить на 3 группы: дети (до 18 лет), взрослые (18 - 50), пожилые (50+). закодировать в виде трёх столбцов с префиксом age_. Старый столбец с age удалить;
    5. Напиток (drink): преобразовать в число 0/1 был ли этот напиток хмельным;
    6. Номер чека (check_number): надо сопоставить со второй таблицей со временем сеанса. И закодировать в виде трёх столбцов, был ли это утренний (morning) сеанс, дневной (day) или вечерний (evening).



In [79]:
import numpy as np
import pandas as pd

df = pd.read_csv('titanic_with_labels.csv', sep=' ', index_col=0)

df = df[(df['sex'] == 'Мужчина') | (df['sex'] == 'M') | (df['sex'] == 'м') | (df['sex'] == 'Ж') | (df['sex'] == 'ж')]
df.reset_index(drop=True, inplace=True)
df['sex'].replace({'Мужчина': 0, 'M': 0, 'м': 0, 'Ж': 1, 'ж': 1}, inplace=True)
df['row_number'].fillna(df['row_number'].max(), inplace=True)
df.loc[(df['liters_drunk'] < 0) | (df['liters_drunk'] > 5), 'liters_drunk'] = np.nan
df['liters_drunk'].fillna(df['liters_drunk'].mean(), inplace=True)

df['age_kid'] = df['age'].apply(lambda x: int(x < 18))
df['age_adult'] = df['age'].apply(lambda x: int(18 <= x <= 50))
df['age_elderly'] = df['age'].apply(lambda x: int(x > 50))

df['drink'] = df['drink'].apply(lambda x: int(("пиво" in x) | ("beer" in x)))

df2 = pd.read_csv("cinema_sessions.csv", sep=' ', index_col=0)
df = df.merge(df2, on='check_number')
df['session_start'] = pd.to_datetime(df['session_start'], format='%H:%M:%S.%f').dt.time
df['morning'] = df['session_start'].apply(lambda x: int(6 <= x.hour < 12))
df['day'] = df['session_start'].apply(lambda x: int(12 <= x.hour < 18))
df['evening'] = df['session_start'].apply(lambda x: int(18 <= x.hour < 24))

print(df)

      sex  row_number  liters_drunk  age  drink  check_number  label  age_kid  \
0       1        79.0           2.0   66      0          3116      0        0   
1       0        23.0           0.0   10      0          8118      0        1   
2       0        92.0           0.0   18      0          2264      1        0   
3       1        52.0           5.0   67      1          7640      1        0   
4       1        57.0           2.0   27      1          8764      0        0   
...   ...         ...           ...  ...    ...           ...    ...      ...   
6665    1       100.0           3.0   15      0          1022      1        1   
6666    0        15.0           3.0   62      1          6510      0        0   
6667    1        11.0           3.0   52      1          9510      1        0   
6668    0        96.0           5.0   72      1          9554      0        0   
6669    1        26.0           0.0   20      0          6198      1        0   

      age_adult  age_elderl